In [14]:
import flwr as fl
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

import numpy as np
import matplotlib.pyplot as plt



In [ ]:
NUM_CLIENTS = 2
BATCH_SIZE = 32


def load_datasets(partition_id: int):
    fds = FederatedDataset(dataset="cifar10", partitioners={"train": NUM_CLIENTS})
    partition = fds.load_partition(partition_id)
    
    # Split dataset: 80% train, 20% test
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)

    def normalize_images(batch):
        """Normalize images to [-1,1] range."""
        batch["img"] = [(np.array(img) / 127.5 - 1.0) for img in batch["img"]]
        return batch

    # Apply transformations (normalization)
    partition_train_test = partition_train_test.with_transform(normalize_images)
    
    def batch_generator(dataset, batch_size):
        """Simple NumPy-based batch generator."""
        data_x, data_y = np.array(dataset["img"]), np.array(dataset["label"])
        for i in range(0, len(data_x), batch_size):
            yield data_x[i : i + batch_size], data_y[i : i + batch_size]

    # Train and validation batch loaders
    trainloader = list(batch_generator(partition_train_test["train"], BATCH_SIZE))
    valloader = list(batch_generator(partition_train_test["test"], BATCH_SIZE))
    
    # Load global test set
    testset = fds.load_split("test").with_transform(normalize_images)
    testloader = list(batch_generator(testset, BATCH_SIZE))

    return trainloader, valloader, testloader


In [16]:
trainloader, _, _ = load_datasets(partition_id=0)
batch = next(iter(trainloader))
images, labels = batch["img"], batch["label"]

# Reshape and convert images to a NumPy array
# matplotlib requires images with the shape (height, width, 3)
images = images.permute(0, 2, 3, 1).numpy()

# Denormalize
images = images / 2 + 0.5

# Create a figure and a grid of subplots
fig, axs = plt.subplots(4, 8, figsize=(12, 6))

# Loop over the images and plot them
for i, ax in enumerate(axs.flat):
    ax.imshow(images[i])
    ax.set_title(trainloader.dataset.features["label"].int2str([labels[i]])[0])
    ax.axis("off")

# Show the plot
fig.tight_layout()
plt.show()

Generating test split: 100%|██████████| 10000/10000 [00:00<00:00, 150081.73 examples/s]
g:\Study\iit kharagpur\ML Lab\Lab 1\ml_lab_venv\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


KeyError: 'img'